In [33]:
import sys  
import os
sys.path.append(os.path.abspath(r"C:\Users\Jil Henry\Documents\Programming\Final Year Project\acne_diagnosis\python"))

In [34]:
import numpy as np
import tensorflow as tf
from data_preprocessing import resize_rescale
import constants as Constants
import data_formatter as df

In [35]:
BATCH_SIZE = 32
IMG_SIZE = 224
MODEL = tf.keras.models.load_model(Constants.MODEL_ABS_PATH)
PARENT_FILE_PATH = './dataset/test/acne/'

In [39]:
files = os.listdir(PARENT_FILE_PATH)
images_array = []
for file in files:
    pil_image = tf.keras.utils.load_img(PARENT_FILE_PATH + file)
    image_array = tf.keras.preprocessing.image.img_to_array(pil_image).tolist()
    images_array.append(image_array)
imgs_np_arr = np.array(images_array)
images_batch = resize_rescale(imgs_np_arr)
images_batch.shape
predictions = MODEL.predict(images_batch)
num_predictions = len(predictions)
type(predictions)

numpy.ndarray

In [48]:
if num_predictions < 1:
    print(df.format_result(
        status='No Prediction',
        result_type=Constants.RESULT_TYPE_CONC))

pred_classes_code = np.array([], dtype='int64')
confidence_list_total = np.zeros_like(predictions[0])

if num_predictions == 1:
    #return{"prediction":str(predictions)}
    pred_class_code, confidence_list = df.format_prediction_result(
            predictions[0])
    #print(np.round(confidence_list.tolist()[pred_class_code],2))
    print( df.format_result(
            status='OK',
            result_type=Constants.RESULT_TYPE_CONC,
            pred_class=Constants.CLASS_NAMES[pred_class_code],
            confidence= confidence_list[pred_class_code],
        ))
elif num_predictions > 1:
    for prediction in predictions:
        pred_class_code, confidence_list = df.format_prediction_result(
                prediction)
        pred_classes_code = np.append(pred_classes_code, pred_class_code)
        confidence_list_total += confidence_list

    print(confidence_list_total)
    # Getting Number of  Most Occurring Class
    pred_classes_code_mode = np.bincount(pred_classes_code)
    max_num = np.max(pred_classes_code_mode)
    pred_classes_code_mode_num = [x for x in pred_classes_code_mode if x == max_num]
    # Return Results
    if len(pred_classes_code_mode_num) > 1:
        print(confidence_list_total)
        print( df.format_result(
              status='Inconclusive',
              result_type=Constants.RESULT_TYPE_INCONC
           ))
    else:
        index = pred_classes_code_mode.argmax()
        confidence = np.round(((confidence_list_total[index] / num_predictions) * 100),2)
        pred_class = Constants.CLASS_NAMES[index]
        print( df.format_result(
                    status='OK',
                    result_type=Constants.RESULT_TYPE_CONC,
                    pred_class=pred_class,
                    confidence=confidence
                ))

0.77
{'status': 'OK', 'resultType': 'Conclusive', 'predClass': 'acne', 'confidence': 0.77}
